In [51]:
 
import os
import subprocess
import json
import pandas as pd
import math

# Dossier vidéos
video_folder = r"C:\Users\loven\Desktop\MTI811-Project\SOURCE"

# Normaliser ratio
def normalize_aspect_ratio(ratio):
    if ratio == 'N/A':
        return ratio
    try:
        width, height = map(int, ratio.split(":"))
        gcd_value = math.gcd(width, height)
        normalized_width = width // gcd_value
        normalized_height = height // gcd_value
        return f"{normalized_width}:{normalized_height}"
    except ValueError:
        return ratio

# Informations vidéo
def get_video_info(video_path):
    try:
        result = subprocess.run(
            ['ffprobe', '-v', 'error', '-print_format', 'json', '-show_streams', '-show_format', video_path],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )
        
        info = json.loads(result.stdout)
        video_stream = next((stream for stream in info['streams'] if stream['codec_type'] == 'video'), None)
        format_info = info['format']
        
        if video_stream:
            resolution = f"{video_stream['width']}x{video_stream['height']}"
            frame_rate = round(eval(video_stream['avg_frame_rate'])) 
            video_format = format_info['format_long_name']
            codec = video_stream['codec_name']
            aspect_ratio = normalize_aspect_ratio(video_stream.get('display_aspect_ratio', 'N/A'))
            duration = float(format_info['duration'])
            bitrate = int(format_info['bit_rate']) // 1000  
            file_size = os.path.getsize(video_path) / (1024 * 1024)  
            chroma_subsampling = video_stream.get('chroma_subsampling', 'N/A')

            return {
                "Nom": os.path.basename(video_path),
                "Résolution": resolution,
                "Taux de trames (fps)": frame_rate,
                "Format": video_format,
                "Codec": codec,
                "Ratio d'aspect": aspect_ratio,
                "Durée (s)": round(duration, 2),
                "Bitrate (kbps)": bitrate,
                "Chroma Subsampling": chroma_subsampling,
                "Taille (MB)": round(file_size, 2)
            }
    except subprocess.CalledProcessError as e:
        print(f"Erreur lors de l'analyse de {video_path}: {e}")
        return None
    except Exception as e:
        print(f"Erreur inconnue avec {video_path}: {e}")
        return None

# Collecter données
video_data = []

# Parcours vidéos
for filename in os.listdir(video_folder):
    if filename.endswith(('.mp4', '.mkv', '.avi', '.mov', 'y4m', '.webm')):
        video_path = os.path.join(video_folder, filename)
        info = get_video_info(video_path)
        if info:
            video_data.append(info)

# Créer tableau
df = pd.DataFrame(video_data)

# Paramétrage affichage
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Affichage
print(df.to_string(index=False, header=True))
 

                        Nom Résolution  Taux de trames (fps)          Format    Codec Ratio d'aspect  Durée (s)  Bitrate (kbps) Chroma Subsampling  Taille (MB)
                bus_cif.y4m    352x288                    30   YUV4MPEG pipe rawvideo      1408:1053       5.00           36496                N/A        21.75
          carphone_qcif.y4m    176x144                    30   YUV4MPEG pipe rawvideo      1408:1053      12.75            9116                N/A        13.85
    claire_qcif-5.994Hz.y4m    176x144                     6   YUV4MPEG pipe rawvideo      1408:1053       5.00            1823                N/A         1.09
            claire_qcif.y4m    176x144                    30   YUV4MPEG pipe rawvideo      1408:1053      16.48            9116                N/A        17.91
    football_qcif_15fps.y4m    176x144                    15   YUV4MPEG pipe rawvideo      1408:1053       8.67            4562                N/A         4.71
           grandma_qcif.y4m    176x144  